<a href="https://colab.research.google.com/github/raviteja1432/NNDL/blob/main/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
import numpy as np
import math
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_excel("/content/drive/MyDrive/loaddata.xlsx")
data.head()

,Date,Time,Load (kW)
0,01.09.2018,00:00:00,5551.82208
1,NaN,01:00:00,4983.17184
2,NaN,02:00:00,4888.39680
3,NaN,03:00:00,5072.95872
4,NaN,04:00:00,5196.25980


In [ ]:
x = []
y = []
for i in range(24 * 7, data.shape[0]):
    x.append(data['Load (kW)'].iloc[i-24 * 7])
    y.append(data['Load (kW)'].iloc[i])
data = pd.DataFrame({'Previous week Load in kw' : x, 'Present week Load in kw' : y})
data.head()

,Previous week Load in kw,Present week Load in kw
0,5551.82208,5346.52812
1,4983.17184,5138.89596
2,4888.39680,4940.77248
3,5072.95872,4766.18688
4,5196.25980,5020.11540


In [ ]:

#normalizing
maxX = np.max(data['Previous week Load in kw'])
minX = np.min(data['Previous week Load in kw'])
maxY = np.max(data['Present week Load in kw'])
minY = np.min(data['Present week Load in kw'])

data['Previous week Load in kw'] = (data['Previous week Load in kw'] - minX) / (maxX - minX)
data['Present week Load in kw'] = (data['Present week Load in kw']- minY) / (maxY - minY)

In [ ]:
from sklearn.model_selection import train_test_split
x = data['Previous week Load in kw']
y = data['Present week Load in kw']
trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.1, random_state=101)

In [ ]:
#Model Training
#Initialization
m = np.random.uniform(-2, 2)
c = np.random.uniform(-2, 2)
eta = 0.5
epochs = 500
gm2 = 0
gc2 = 0
e = 1E-6

# Loop for epochs
for _ in range(epochs):
    
    for i in range(trainX.shape[0]):
        # Calculating Grad M
        gradM = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c) * trainX.iloc[i]
    
        # Calculating Grad C
        gradC = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c)
        
        # Calculating updated values of gm2 and gc2
        gm2 += (gradM ** 2)
        gc2 += (gradC ** 2)
        
        # Updating m and c values
        m -= (eta * gradM) / ((e + gm2) ** 0.5)
        c -= (eta * gradC) / ((e + gc2) ** 0.5)

# Print model parameters after training
print(f'm = {m}\nc = {c}')

m = 0.48573611202736794
c = 0.23135673647212723


In [ ]:


train_pred = [m * trainX.iloc[i] + c for i in range(trainX.shape[0])]
test_pred = [m * testX.iloc[i] + c for i in range(testX.shape[0])]

# Denormalization of Output Values
train_pred = [i * (maxY - minY) + minY for i in train_pred]
test_pred = [i * (maxY - minY) + minY for i in test_pred]
trainY = [i * (maxY - minY) + minY for i in trainY]
testY = [i * (maxY - minY) + minY for i in testY]
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error
rmse = math.sqrt(mean_squared_error(train_pred, trainY)) 
print("testing error")
print('Root mean square error', rmse) 
mse = (mean_squared_error(train_pred,trainY)) 
print('Mean square error', mse) 
mae=mean_absolute_error(train_pred,trainY)
print('Mean absolute error', mae)


#Testing Accuracies
rmse = math.sqrt(mean_squared_error(test_pred,testY ))
print("testing error") 
print('Root mean square error', rmse) 
mse = (mean_squared_error(test_pred,testY))
print('Mean square error', mse) 
mae=mean_absolute_error(test_pred,testY)
print('Mean absolute error', mae)

testing error
Root mean square error 944.1524919476075
Mean square error 891423.9280508771
Mean absolute error 737.0269996055231
testing error
Root mean square error 983.9278449764412
Mean square error 968114.0041199836
Mean absolute error 787.1165728920556


In [ ]:

#checking the difference between actal value and predicted value of training data
pd.DataFrame({'Actual value' : trainY, 'Predicted value' : train_pred})

,Actual value,Predicted value
0,5086.98792,5060.579638
1,6378.60960,5731.527345
2,7647.78456,6182.159341
3,6165.05400,5656.068457
4,5918.45184,5501.817155
...,...,...
1809,5020.11540,5866.686639
1810,4048.82712,5524.166926
1811,5674.03200,5578.185186
1812,5621.65632,6995.539488


In [ ]:


#checking the difference between actal value and predicted value of testing data
pd.DataFrame({'Actual value' : testY, 'Predicted value' : test_pred})

,Actual value,Predicted value
0,7609.43808,6756.056008
1,4187.24856,5748.801067
2,6738.69240,6181.553246
3,5640.67368,6206.100113
4,6550.07760,5442.419797
...,...,...
197,5761.79244,6598.849990
198,6955.67736,6087.153873
199,8607.69360,6021.089465
200,6532.93080,6219.509976


In [ ]:
#testing for real example
x = float(input('Enter the load of previous week at present hour and day : '))
x = (x - minX) / (maxX - minX)
prediction = m * x + c
prediction = (prediction * (maxY - minY)) + minY
print('Predicted load at present hour :', prediction)

Enter the load of previous week at present hour and day : 5467.9
Predicted load at present hour : 5656.191479914067
